In [1]:
import SNIDsn
import SNIDdataset as snid
import numpy as np
import SNePCA

import plotly.plotly as ply
import plotly.graph_objs as go
import plotly.tools as tls

import matplotlib.pyplot as plt
from BinSpectra import lowres_dataset
from BinSpectra import GetTypes
import warnings
from BinSpectra import random_noise_dataset
from BinSpectra import replace_flux_with_noise
warnings.filterwarnings('ignore')

from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

PATH = '../Data/DataProducts/'

In [13]:
def loaddata(phase):
    """
    Loads the pickled dataset
    """
    dsname = "dataset{}.pickle".format(phase)
    return snid.loadPickle(PATH + dsname)


def get_bin_length(phase, bin_factor):
    dataset = loaddata(phase)
    dataset_key_list = list(dataset.keys())
    wvl = dataset[dataset_key_list[0]].wavelengths
    wbin = (np.log(wvl[1]) - np.log(wvl[0])) * bin_factor
    return wbin


def LinearSVM(phase, pcax, pcay, dphase=5, noise=False):
    dataset = loaddata(phase)
    if noise:
        dataset = replace_flux_with_noise(dataset)
        dataset_lowres = lowres_dataset(dataset, k)
    else:
        dataset_lowres = lowres_dataset(dataset, k)
    snidPCA_ph = SNePCA.SNePCA(dataset_lowres, phase - dphase, phase + dphase)
    snidPCA_ph.snidPCA()
    snidPCA_ph.calcPCACoeffs()
    
    IIbMask, IbMask, IcMask, IcBLMask = snidPCA_ph.getSNeTypeMasks()
    labels = 1*IIbMask + 2*IbMask + 3*IcMask + 4*IcBLMask
    
    x = snidPCA_ph.pcaCoeffMatrix[:,pcax-1]
    y = snidPCA_ph.pcaCoeffMatrix[:,pcay-1]
    data = np.column_stack((x,y))
    
    linsvm = LinearSVC()
    test_score = []
    
    for i in range(50):
        trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.30)
        linsvm.fit(trainX, trainY)
        trainscore = linsvm.score(trainX, trainY)
        testscore = linsvm.score(testX, testY)    
        test_score.append(testscore)
    av_svm_sc, svm_std = np.array(test_score).mean(), np.array(test_score).std()
    return av_svm_sc, svm_std

# 1 Real Spectrum

## 1.1 Phase 0: PC1 vs PC5

### 1.1.1 Full resolution

In [26]:
k = get_bin_length(0, 1)
#k = get_bin_length(0, 82)
LinearSVM(0, 1, 5)

(0.5529411764705884, 0.1158689153152483)

### 1.1.2 Lowest resolution

In [40]:
#k = get_bin_length(0, 1)
k = get_bin_length(0, 82)
LinearSVM(0, 1, 5)

(0.5305882352941176, 0.08523527381967039)

## 1.2 Phase 5: PC1 vs PC3

### 1.2.1 Full resolution

In [34]:
k = get_bin_length(5, 1)
#k = get_bin_length(5, 82)
LinearSVM(5, 1, 3)

(0.585, 0.12484990989183774)

### 1.2.2 Lowest resolution

In [41]:
#k = get_bin_length(5, 1)
k = get_bin_length(5, 82)
LinearSVM(5, 1, 3)

(0.57, 0.08536685539481936)

## 1.3 Phase 10: PC1 vs PC3

### 1.3.1 Full resolution

In [51]:
k = get_bin_length(10, 1)
#k = get_bin_length(10, 82)
LinearSVM(10, 1, 3)

(0.7105882352941175, 0.10040747432165584)

### 1.3.2 Lowest resolution

In [45]:
#k = get_bin_length(10, 1)
k = get_bin_length(10, 82)
LinearSVM(10, 1, 3)

(0.5494117647058823, 0.08773189735295134)

## 1.4 Phase 15: PC1 vs PC3

### 1.4.1 Full resolution

In [49]:
k = get_bin_length(15, 1)
#k = get_bin_length(15, 82)
LinearSVM(15, 1, 3)

(0.6835294117647058, 0.10646246312838774)

### 1.4.2 Lowest resolution

In [54]:
#k = get_bin_length(15, 1)
k = get_bin_length(15, 82)
LinearSVM(15, 1, 3)

(0.41529411764705876, 0.09730627991412806)

# 2 Random Noise

## 2.1 Phase 0: PC1 vs PC5

### 2.1.1 Full resolution

In [91]:
k = get_bin_length(0, 1)
#k = get_bin_length(0, 82)
for i in range(10):
    svm_scores_0 = LinearSVM(0, 1, 5, noise=True)
    print(svm_scores_0)

(0.19058823529411764, 0.08027633933756227)
(0.20588235294117646, 0.08086898285216189)
(0.13294117647058823, 0.078753261040845)
(0.18, 0.07000741433793807)
(0.3, 0.09647776156974544)
(0.15058823529411763, 0.06671739707650741)
(0.23529411764705885, 0.09701425001453319)
(0.16941176470588232, 0.07860373218112972)
(0.14823529411764705, 0.07555103227295211)
(0.16823529411764707, 0.07253205949081577)


### 2.1.2 Lowest resolution

In [67]:
#k = get_bin_length(0, 1)
k = get_bin_length(0, 82)
LinearSVM(0, 1, 5, noise=True)

(0.1564705882352941, 0.07404291283557442)

## 2.2 Phase 5: PC1 vs PC3

### 2.2.1 Full resolution

In [92]:
k = get_bin_length(5, 1)
#k = get_bin_length(5, 82)
for i in range(10):
    svm_scores_5 = LinearSVM(5, 1, 3, noise=True)
    print(svm_scores_5)

(0.22375, 0.08300037650517014)
(0.1775, 0.07110731326663945)
(0.155, 0.07628073151196178)
(0.1375, 0.06846531968814576)
(0.3275, 0.11357816691600547)
(0.20375, 0.08822024994296945)
(0.25125, 0.08148044243866133)
(0.18625, 0.0972191467767538)
(0.27, 0.07949056547792323)
(0.21125, 0.08186307165016471)


### 2.2.2 Lowest resolution

In [74]:
#k = get_bin_length(5, 1)
k = get_bin_length(5, 82)
LinearSVM(5, 1, 3, noise=True)

(0.16875, 0.08315084184781293)

## 2.3 Phase 10: PC1 vs PC3 

### 2.3.1 Full resolution

In [93]:
k = get_bin_length(10, 1)
#k = get_bin_length(10, 82)
for i in range(10):
    svm_scores_10 = LinearSVM(10, 1, 3, noise=True)
    print(svm_scores_10)

(0.31529411764705884, 0.09755488207824885)
(0.1811764705882353, 0.08712239332505232)
(0.27764705882352947, 0.0903970480578755)
(0.26, 0.10659239051763017)
(0.22117647058823534, 0.08769244778157283)
(0.3423529411764705, 0.07596211038756671)
(0.1835294117647059, 0.07591654496424319)
(0.2564705882352942, 0.10506876970242873)
(0.24235294117647066, 0.08120204200817599)
(0.2341176470588235, 0.08441945035722465)


### 2.3.2 Lowest resolution

In [76]:
#k = get_bin_length(10, 1)
k = get_bin_length(10, 82)
LinearSVM(10, 1, 3, noise=True)

(0.21176470588235294, 0.08645257915705334)

## 2.4 Phase 15: PC1 vs PC3

### 2.4.1 Full resolution

In [94]:
k = get_bin_length(15, 1)
#k = get_bin_length(15, 82)
for i in range(10):
    svm_scores_15 = LinearSVM(15, 1, 3, noise=True)
    print(svm_scores_15)

(0.2, 0.08401680504168059)
(0.30000000000000004, 0.0866524697803308)
(0.2282352941176471, 0.0853569748104332)
(0.26, 0.09849384783976986)
(0.28, 0.08198235948322045)
(0.1623529411764706, 0.06396365750149735)
(0.21764705882352942, 0.0858501148195672)
(0.1894117647058824, 0.09058823529411765)
(0.22705882352941173, 0.0888293726165302)
(0.2788235294117647, 0.10377644324706055)


### 2.4.2 Lowest resolution

In [80]:
#k = get_bin_length(15, 1)
k = get_bin_length(15, 82)
LinearSVM(15, 1, 3, noise=True)

(0.21647058823529408, 0.06803215197553393)